__Author__: Bogdan Bintu

__Email__: bbintu@g.harvard.edu

__Date__:3/4/2020

### Trans A/B density ratio at transcribing vs silent

In [2]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.size']=15
matplotlib.rcParams['font.family']='Arial'
import matplotlib.pylab as plt
import numpy as np
import os

### Load data

In [4]:
folder = r'\Data'
fid = open(folder+os.sep+r'genomic-scale-with-transcription.tsv','r')
lines = np.array([ln[:-1].split('\t')for ln in fid if len(ln)>0])
zxy = np.array(lines[1:,:3][:],dtype=np.float)
zxy = zxy.reshape([-1,2082,3])/1000 #transform to um

In [11]:
Transcr = np.array(['on' in e for e in lines[1:,8]])
Transcr_WT = Transcr.reshape([-1,1041])

### Load density

In [23]:
#Please see Part 3 how to calculate densities
resWT = np.load(r'Data\densityIMR90Untreated.npy')
keep = slice(None)
AD = resWT[keep,0,0,:]
BD = resWT[keep,1,0,:]
Dratio_WT = AD/BD
Dratio_WT = Dratio_WT.reshape([-1,1041])[-len(Transcr_WT):]

good_WT = (~np.isnan(zxy[:,:,0])).reshape([-1,1041])

C:\Users\Bogdan\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in divide
  after removing the cwd from sys.path.


### A/B density as a function of transcriptional state

In [ ]:
Transcr =Transcr_WT==1
DratioT = Dratio_WT.copy()
DratioT[~(Transcr&good_WT)]=np.nan
dT = np.nanmedian(DratioT,0)
DratioS = Dratio_WT.copy()
DratioS[~((~Transcr)&good_WT)]=np.nan
dS = np.nanmedian(DratioS,0)
logTS = np.log(dT/dS)/np.log(2)
nanlogTS = logTS[~np.isnan(logTS)]
nanlogTS_sort = np.sort(nanlogTS)
fig = plt.figure()
fr=1.*np.sum(nanlogTS_sort>0)/len(nanlogTS_sort)
plt.title('Trascribing/Silent\n'+str(np.round(fr,2))+' increase')
plt.plot(nanlogTS_sort,'o',color='orange')
plt.plot([0,len(nanlogTS_sort)],[0,0],lw=4,color='k')
plt.ylabel('Log2 difference of \ntrans A/B density ratio')
plt.xlabel('Locus index')

#folder = r'C:\Users\Bogdan\Dropbox\2020 Chromatin Imaging Manuscript\Revision\FinalFigures\Figure 7\Subpanels'
#import os
#fig.savefig(folder+os.sep+r'Fig7BTranscription_final.pdf')

### Transcriptional state as a function of A/B density

In [ ]:
Transcr =Transcr_WT==1
perc_75 = np.nanpercentile(Dratio_WT,75,axis=0)
perc_25 = np.nanpercentile(Dratio_WT,25,axis=0)
Tr_high = (Transcr&good_WT)
keep = (Dratio_WT>perc_75)&good_WT
Tr_high[~keep] = 0
fr_high = 1.*np.sum(Tr_high,axis=0)/np.sum(keep,axis=0)

Tr_low = (Transcr&good_WT)
keep = (Dratio_WT<perc_25)&good_WT
Tr_low[~keep] = 0
fr_low = 1.*np.sum(Tr_low,axis=0)/np.sum(keep,axis=0)


logTS = np.log(fr_high/fr_low)/np.log(2)
nanlogTS = logTS[~np.isnan(logTS)]
nanlogTS_sort = np.sort(nanlogTS)

fr=1.*np.sum(nanlogTS_sort>0)/len(nanlogTS_sort)

fig = plt.figure()
plt.title('High/low trans A/B density ratio\n'+str(np.round(fr,2))+' increase')
plt.plot(nanlogTS_sort,'o',color='orange')
plt.plot([0,len(nanlogTS_sort)],[0,0],lw=4,color='k')
plt.ylabel('Log2 difference of firing rate')
plt.xlabel('Locus index')

#folder = r'C:\Users\Bogdan\Dropbox\2020 Chromatin Imaging Manuscript\Revision\FinalFigures\Figure 7\Subpanels'
#import os
#fig.savefig(folder+os.sep+r'Fig7CTranscriptionIn25pecr75percDensity_final.pdf')